# Проект: pretraining LLM и posttraining LLM

## Постановка задачи

- **Pretraining**
    - Претрейн — самый ресурсоёмкий этап обучения LLM. Чтобы полноценно обучить даже небольшую модель (менее 1B), понадобится более 10к GPU-часов на A100. Чтобы не тратить недели на обучение, но отработать ключевые приёмы, в проекте вы выполните упрощённую задачу. 
    - При полноценном претрейне модель учится обобщать знания из данных, на которых происходило обучение, чтобы потом извлекать эти знания по текстовым запросам уже после обучения. Упростим задачу — научим модель только структуре языка. 
    - Сосредоточимся на одном узком домене — **текстах произведений русской литературы** — и **обучим модель продолжать фразы из этого домена разумным текстом**. 
- **Posttraining**
    - Для SFT-этапа можно использовать значительно меньше данных, поэтому возьмём модель крупнее. Рассмотрим базовую модель **Qwen2.5-0.5B**, с которой вы встречались в уроках. **Обучите её генерировать ответы на инструктивные русскоязычные вопросы**.

## Критерии успеха
- **Pretraining** - **чтобы оценить качество, используйте промпты**:
    - "Все мысли, которые имеют огромные последствия",
    - "Сила войска зависит от его духа",
    - "Мысль о том, что он принес страдания",
    - "Человек сознает себя свободным",
    - "Что бы ни случилось, я всегда буду",
    - "Любовь мешает смерти",
    - "Нет, жизнь не кончена",
    - "Всякая мысль, даже самая простая",
    - "Война не любезность, а самое гадкое дело",
    - "Чтобы жить честно"
- **Posttraining** - Факты могут быть ошибочными, но **язык ответа должен быть русским и должна сохраняться структура ответа**. Качество данных **должно быть сопоставимо с таким вариантом**:
    - Model Input 1:
    сколько планет в нашей солнечной системе?
        - Model Output 1:
    Согласно последним исследованиям, Солнце имеет 8 планеты. Это дает планетам, которые соответствуют разным критериям, таким как диета, местоположение, плотность, климат и другие.
    assistant
    Общее размер нашей Солнечной системы составляет примерно 9,9 миллиарда километров, а в результате на каждом из них есть планета,
    - Model Input 2:
    расскажи стих
        - Model Output 2:
    Вот сладкий сладкий вкус, который ты получаешь на себе
    Когда я говорю, что я люблю тебя
    Мое сердце, у меня есть все, что я хочу, чтобы сделать
    У меня есть все, что мне нужно, чтобы быть хорошим
    Так давай пойдем в ресторан
    assistant
    Пока мы уседим, что мы можем
    И в этот момент мы возвращаемся к при
    - Model Input 3:
    когда собирать крыжовник?
        - Model Output 3:
    Когда собираешь крыжовник, важно не беспокоиться о том, что другие могут его хвастаться. Вместо этого проверяйте свои способности и готовность. Вы можете изучить свой талант и стремления, прежде чем принимать решение, и признавать, что у вас есть свои сильные стороны, а также слабые места. Не снимайте крыжовник в обществе, и обязательно помните о
    - Model Input 4:
    Как быстро выучить новый язык?
        - Model Output 4:
    Скорость усвоения нового языка может сильно различаться в зависимости от нескольких факторов, таких как скорость, с которой вы пакетируете информацию через ее и ее способность учиться в различных условиях. Тем не менее, вот некоторые общие рекомендации: 1. Делайте сценарии. Обычно это может занять от 2 до 4 лет, чтобы понять основные концепции. Вы можете начать с

<div class="alert alert-secondary" style="background-color:#D9EEE1;color:black;">

## Описание данных

- Pretrain
    - https://github.com/JoannaBy/RussianNovels/tree/master/corpus 
- Posttrain
    - русскоязычный инструктивный датасет [d0rj/alpaca-cleaned-ru](https://huggingface.co/datasets/d0rj/alpaca-cleaned-ru) в диалоговом формате

## Инициализация

In [1]:
!nvidia-smi

Fri Oct 31 01:17:23 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.107.02             Driver Version: 550.107.02     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla V100-SXM2-32GB           On  |   00000000:00:05.0 Off |                    0 |
| N/A   36C    P0             62W /  300W |       5MiB /  32768MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

### Импорт библиотек

In [2]:
import os
import re
from git import Repo
from pathlib import Path
from datasets import Dataset, DatasetDict, load_dataset
import pandas as pd
from tokenizers import Tokenizer, pre_tokenizers, trainers
from transformers import (
    LlamaConfig,
    LlamaForCausalLM,
    PreTrainedTokenizerFast,
    Trainer,
    TrainingArguments,
    TrainerCallback,
    DataCollatorForLanguageModeling,
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainerCallback,
    TrainerState,
    TrainerControl
)
from tokenizers.models import BPE
from tokenizers.decoders import ByteLevel as ByteLevelDecoder
from transformers import LlamaConfig, LlamaForCausalLM
from transformers import Trainer, TrainingArguments
import torch
import accelerate
from trl import SFTTrainer, SFTConfig
#from peft import LoraConfig
import gc

/home/user/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Установка главных параметров

In [3]:
JUST_SMOKE_TEST = False  # True = временно для отладки на малом дaтасете перед GPU

PRETRAIN_DATASET =  "pretrain-dataset"
DICT_SIZE = 3000
SPECIAL_TOKEN = "<|endoftext|>"
CONTEXT_LENGTH = 512

SFT_DATASET = "d0rj/alpaca-cleaned-ru"
SFT_MODEL_NAME = "Qwen/Qwen3-1.7B"
FINETUNED_MODEL_PATH = "full_sft.pkl"

# Pretraining

<div class="alert alert-secondary">


Скачайте данные из репозитория и упакуйте их в один датасет. Вам понадобятся все произведения из репозитория.

Проведите препроцессинг данных:
- Очистите их от дубликатов.
- Очистите от предложений с буквами не из кириллицы.
- Обработайте повторяющуюся пунктуацию и т. д.
- Разбейте на чанки поменьше, чтобы можно было добавить <bos> и <eos> токены в соответствии с обучаемой длиной контекста.

In [4]:
if not os.path.isdir(PRETRAIN_DATASET):
    Repo.clone_from("https://github.com/JoannaBy/RussianNovels.git", PRETRAIN_DATASET) 

In [5]:
texts = []
for file_path in Path(Path(PRETRAIN_DATASET, "corpus")).glob("*.txt"):
    with file_path.open("r", encoding="utf-8") as f:
        for line in f:
            texts.append(line.strip())

# удалим строки с недопустимыми символами
allowed_pattern = re.compile(r"^[А-Яа-яЁё0-9.,!?;:\-()\"'\s]+$")
texts = [t for t in texts if allowed_pattern.fullmatch(t)]

# Заменяем повторяющиеся знаки пунктуации на один
texts = [re.sub(r'([.,!?;:\-()])\1+', r'\1', t) for t in texts]

#удалим дубликаты
texts = pd.DataFrame({'t': texts}).drop_duplicates()['t'].tolist()

#перенесём строки, если они длиннее 300 слов
max_words = 300 # из рассчёта, что ~1.5 токена на слово, а токенов у нас в контексте 512
result = []
for text in texts:
    words = text.split()
    if len(words) > max_words:
        parts = [' '.join(words[i:i + max_words]) \
                    for i in range(0, len(words), max_words)]
        result.extend(parts)
    else:
        result.append(text)
texts = result

In [6]:
TEST_TEXT = texts[100]
TEST_TEXT

'Генерал покраснел ужасно, Коля тоже покраснел и стиснул себе руками голову; Птицын быстро отвернулся. Хохотал попрежнему один только Фердыщенко. Про Ганю и говорить было нечего: он всё время стоял, выдерживая немую и нестерпимую муку.'

<div class="alert alert-secondary">

Создайте и обучите собственный токенизатор на полученных данных. Размер словаря выберите небольшим: при обучении только на рассмотренных текстах — около 3к токенов. В рассматриваемых данных язык намного менее разнообразен, чем в совокупных данных, поэтому крупные токенизаторы от реальных LLM могут не подойти. 
При создании токенизатора можете ориентироваться на [материал huggingface.co](https://huggingface.co/learn/llm-course/ru/chapter6/8). Рекомендуем использовать BPE.

In [7]:
tokenizer = Tokenizer(BPE())
tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False)
tokenizer.pre_tokenizer.pre_tokenize_str(TEST_TEXT)
tokenizer.decoder = ByteLevelDecoder()

In [8]:
def get_training_corpus(texts):
    for i in range(0, len(texts), 1000):
        batch = [texts[j] for j in range(i, min(i + 1000, len(texts)))]
        yield batch

In [9]:
%%time
trainer = trainers.BpeTrainer(vocab_size=DICT_SIZE, special_tokens=[SPECIAL_TOKEN])
tokenizer.train_from_iterator(get_training_corpus(texts), trainer=trainer)




CPU times: user 1min 37s, sys: 16.9 s, total: 1min 54s
Wall time: 19.3 s


Посмотрим на токенизатор:

In [10]:
print("Vocab size:", tokenizer.get_vocab_size())
print("Sample vocab:", list(tokenizer.get_vocab().keys())[:30])

Vocab size: 3000
Sample vocab: ['ÑĨ', 'ĠÑģÑĤÐ¾Ð¸ÑĤ', 'ĠÑģÐ°Ð¼Ð¾Ð¼', 'ÑıÐ½', 'ĠÑģÐµÑĢÑĮÐµÐ·', 'ĠÑģÐ¼ÐµÑħ', 'ĠÐ¢ÑĥÑĤ', 'ÑģÐºÐ¾Ð»ÑĮÐºÐ¾', 'ĠÐ»ÑİÐ±Ð¾Ð²ÑĮ', 'Ð½Ð¾ÑĪ', 'Ð¸Ð¾Ð½', 'ĠÑĪÐ°ÑĢ', 'ĠÐ²ÑĭÑĪÐ»Ð°', 'ÐĶ', 'ĠÐ½', 'ĠÑĢÐ°Ð·Ð²', 'Ð´Ñĭ', 'ĠÐ¶ÐµÐ½Ð°', 'Ð»Ð¸Ñĩ', 'ĠÐ¾Ð³Ð»Ñı', 'Ð»Ð¸Ð·', 'ĠÐ¾Ð±', 'ĠÐŀÐ´', 'ÑģÑĤÑĢÐµÐ»', 'ĠÐ²Ð½Ð¸Ð·', 'ĠÐĵÐ´Ðµ', 'ÐµÐ»Ð¾', 'Ð»Ð¸ÑĪ', 'ĠÑģÐ¼Ðµ', 'ĠÐĲÐ½']


Посмотрим на токенизированный текст:

In [11]:
print (TEST_TEXT)
encoding = tokenizer.encode(TEST_TEXT)
print (encoding.tokens)

Генерал покраснел ужасно, Коля тоже покраснел и стиснул себе руками голову; Птицын быстро отвернулся. Хохотал попрежнему один только Фердыщенко. Про Ганю и говорить было нечего: он всё время стоял, выдерживая немую и нестерпимую муку.
['Ðĵ', 'ÐµÐ½ÐµÑĢ', 'Ð°Ð»', 'ĠÐ¿Ð¾Ðº', 'ÑĢÐ°Ñģ', 'Ð½ÐµÐ»', 'ĠÑĥÐ¶Ð°Ñģ', 'Ð½Ð¾', ',', 'ĠÐļ', 'Ð¾Ð»Ñı', 'ĠÑĤÐ¾Ð¶Ðµ', 'ĠÐ¿Ð¾Ðº', 'ÑĢÐ°Ñģ', 'Ð½ÐµÐ»', 'ĠÐ¸', 'ĠÑģÑĤ', 'Ð¸Ñģ', 'Ð½ÑĥÐ»', 'ĠÑģÐµÐ±Ðµ', 'ĠÑĢÑĥÐºÐ°Ð¼Ð¸', 'ĠÐ³Ð¾Ð»Ð¾Ð²Ñĥ', ';', 'ĠÐŁ', 'ÑĤÐ¸', 'ÑĨÑĭ', 'Ð½', 'ĠÐ±ÑĭÑģÑĤÑĢÐ¾', 'ĠÐ¾ÑĤÐ²', 'ÐµÑĢ', 'Ð½ÑĥÐ»ÑģÑı', '.', 'ĠÐ¥', 'Ð¾ÑħÐ¾ÑĤ', 'Ð°Ð»', 'ĠÐ¿Ð¾Ð¿', 'ÑĢÐµÐ¶', 'Ð½ÐµÐ¼Ñĥ', 'ĠÐ¾Ð´Ð¸Ð½', 'ĠÑĤÐ¾Ð»ÑĮÐºÐ¾', 'ĠÐ¤', 'ÐµÑĢ', 'Ð´Ñĭ', 'Ñī', 'ÐµÐ½', 'ÐºÐ¾', '.', 'ĠÐŁÑĢ', 'Ð¾', 'ĠÐĵ', 'Ð°Ð½', 'Ñİ', 'ĠÐ¸', 'ĠÐ³Ð¾Ð²Ð¾ÑĢÐ¸ÑĤÑĮ', 'ĠÐ±ÑĭÐ»Ð¾', 'ĠÐ½ÐµÑĩÐµÐ³Ð¾', ':', 'ĠÐ¾Ð½', 'ĠÐ²ÑģÑĳ', 'ĠÐ²ÑĢÐµÐ¼Ñı', 'ĠÑģÑĤÐ¾ÑıÐ»', ',', 'ĠÐ²Ñĭ', 'Ð´ÐµÑĢÐ¶', 'Ð¸Ð²Ð°Ñı', 'ĠÐ½ÐµÐ¼', 'ÑĥÑİ', 'ĠÐ¸', 'ĠÐ½Ðµ', 'ÑģÑĤ', 'ÐµÑĢÐ¿', 'Ð¸Ð¼', 'ÑĥÑİ', 'ĠÐ¼', 'Ñĥ', 'ÐºÑĥ', '.']


Посмотрим на токены м 0-го по 10-й:

In [ ]:
for i in range (10):
    start, end = encoding.offsets[i]
    TEST_TEXT[start:end]

In [13]:
wrapped_tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=tokenizer,
    bos_token=SPECIAL_TOKEN,
    eos_token=SPECIAL_TOKEN,
    unk_token="<unk>",
    pad_token=SPECIAL_TOKEN # Зададим пэддинг-токен как токен конца
    ) 

<div class="alert alert-secondary">

Токенизируйте данные и подготовьте их к претрейну с длиной контекста 512 токенов в виде экземпляра класса transformers.Dataset.

In [14]:
texts = [text + SPECIAL_TOKEN for text in texts]

In [15]:
texts[:5]

['Федор Михайлович Достоевский<|endoftext|>',
 'Идиот<|endoftext|>',
 'ИДИОТ<|endoftext|>',
 'ЧАСТЬ ПЕРВАЯ.<|endoftext|>',
 'И повел плечами.<|endoftext|>']

In [16]:
%%time

if JUST_SMOKE_TEST:
    texts = texts[:100]

encodings = wrapped_tokenizer(
    texts,
    add_special_tokens=False,
    truncation=False,
)
all_ids = []
for ids in encodings["input_ids"]:
    all_ids.extend(ids)

chunks = [
    all_ids[i:i + CONTEXT_LENGTH]
    for i in range(0, len(all_ids), CONTEXT_LENGTH)
    if len(all_ids[i:i + CONTEXT_LENGTH]) == CONTEXT_LENGTH
]

attention_masks = [[1] * CONTEXT_LENGTH for _ in chunks]

dataset = Dataset.from_dict({
    "input_ids": chunks,
    "attention_mask": attention_masks,
})

dataset

CPU times: user 1min 24s, sys: 10 s, total: 1min 34s
Wall time: 21.6 s


Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 22335
})

<div class="alert alert-secondary">

Инициализируйте модель ~150M параметров c произвольной decoder-only архитектурой трансформера. Например, можно рассмотреть LlamaConfig с параметрами:
- hidden_size=1024, intermediate_size=1536, num_hidden_layers=16, num_attention_heads=16, num_key_value_heads=8.

Подготовьте коллбэки для валидации качества на промптах. Реализуйте обучение с помощью Trainer. Обратите внимание на параметры регуляризации weight_decay. Используйте подходящий batch_size — в диапазоне 64—128.

In [17]:
test_prompts = [
    "Все мысли, которые имеют огромные последствия",
    "Сила войска зависит от его духа",
    "Мысль о том, что он принес страдания",
    "Человек сознает себя свободным",
    "Что бы ни случилось, я всегда буду",
    "Любовь мешает смерти",
    "Нет, жизнь не кончена",
    "Всякая мысль, даже самая простая",
    "Война не любезность, а самое гадкое дело",
    "Чтобы жить честно"
]

In [18]:
gc.collect()
torch.cuda.empty_cache()
torch.cuda.synchronize()
torch.cuda.ipc_collect()

In [19]:
print(torch.cuda.memory_summary())

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |      0 B   |      0 B   |      0 B   |      0 B   |
|       from large pool |      0 B   |      0 B   |      0 B   |      0 B   |
|       from small pool |      0 B   |      0 B   |      0 B   |      0 B   |
|---------------------------------------------------------------------------|
| Active memory         |      0 B   |      0 B   |      0 B   |      0 B   |
|       from large pool |      0 B   |      0 B   |      0 B   |

In [20]:
config = LlamaConfig(
    vocab_size=wrapped_tokenizer.vocab_size,
    hidden_size=1024,
    intermediate_size=1536,
    num_hidden_layers=16,
    num_attention_heads=16,
    num_key_value_heads=8,
    max_position_embeddings=512,
    bos_token_id=wrapped_tokenizer.bos_token_id,
    eos_token_id=wrapped_tokenizer.eos_token_id,
)

model = LlamaForCausalLM(config)
print(f"Модель инициализирована: {sum(p.numel() for p in model.parameters())/1e6:.1f}M параметров")

dataset = dataset.map(lambda x: {"labels": x["input_ids"]})

dataset = dataset.train_test_split(test_size=0.05, seed=42)
train_dataset = dataset["train"]
val_dataset = dataset["test"]

collator = DataCollatorForLanguageModeling(
    tokenizer=wrapped_tokenizer,
    mlm=False
)

class PromptEvalCallback(TrainerCallback):
    def __init__(self, tokenizer, prompts):
        self.tokenizer = tokenizer
        self.prompts = prompts

    def on_evaluate(self, args, state, control, **kwargs):
        model = kwargs["model"]
        model.eval()
        print("\n=== Валидация на промптах ===")
        for prompt in self.prompts:
            inputs = self.tokenizer(prompt, return_tensors="pt").to(model.device)
            with torch.no_grad():
                outputs = model.generate(**inputs, max_new_tokens=30)
            text = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
            print(f"\n[Prompt] {prompt}\n[Output] {text}")

callback = PromptEvalCallback(wrapped_tokenizer, test_prompts)

training_args = TrainingArguments(
    output_dir="./llama-small-pretrain",
    per_device_train_batch_size=8,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    learning_rate=2e-4,
    weight_decay=0.01,
    logging_steps=50,
    #evaluation_strategy="steps",
    do_eval=True,
    eval_steps=200,
    save_steps=200,
    save_total_limit=2,
    report_to="none",
    bf16=torch.cuda.is_available()
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collator,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=wrapped_tokenizer,
    callbacks=[callback],
)

Модель инициализирована: 132.0M параметров


Map: 100%|██████████| 22335/22335 [00:13<00:00, 1632.27 examples/s]
/tmp/ipykernel_6843/1803388242.py:62: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [21]:
%%time
trainer.train()

Step,Training Loss
50,6.755300
100,6.045600
150,5.588100
200,5.277700
250,4.972900
300,4.686700
350,4.452000
400,4.295600
450,4.180900
500,4.081800


CPU times: user 18min 11s, sys: 3min 37s, total: 21min 48s
Wall time: 21min 48s


TrainOutput(global_step=664, training_loss=4.76903930342341, metrics={'train_runtime': 1306.6406, 'train_samples_per_second': 16.239, 'train_steps_per_second': 0.508, 'total_flos': 8404196237770752.0, 'train_loss': 4.76903930342341, 'epoch': 1.0})

Сгенерируйте ответы на запросы test_prompts. Чтобы ревьюер мог оценить результаты, оставьте генерацию в ноутбуке.

In [22]:
model.eval()
model.to("cuda" if torch.cuda.is_available() else "cpu")

for prompt in test_prompts:
    inputs = wrapped_tokenizer(prompt, return_tensors="pt").to(model.device)
    
    # Удаляем token_type_ids если они есть
    if 'token_type_ids' in inputs:
        del inputs['token_type_ids']
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=50,
            temperature=0.8,
            top_p=0.95,
            do_sample=True,
            pad_token_id=wrapped_tokenizer.pad_token_id,
            eos_token_id=wrapped_tokenizer.eos_token_id,
        )

    text = wrapped_tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(f"Prompt: {prompt}\nОтвет модели: {text}")

Prompt: Все мысли, которые имеют огромные последствия
Ответ модели: Все мысли, которые имеют огромные последствия, как сомнения и не могла быть отдаст с этой силы, но что она устройство. Он находил из своей туманских отрядов, в том, что он уже хотел в то, что он
Prompt: Сила войска зависит от его духа
Ответ модели: Сила войска зависит от его духа; он, сорвавшись, подтвердила ему, и, входя в лицо, закрывал в дверь, и, покорясь, влюбленные на стол, по-французски с пухлой,
Prompt: Мысль о том, что он принес страдания
Ответ модели: Мысль о том, что он принес страдания на него. Он так, как он говорил. Он как будто в это время участвовал в эту минуту, он чувствовал, что он уже не умеющиеся на свете, что было в нем. Всякий день в нем
Prompt: Человек сознает себя свободным
Ответ модели: Человек сознает себя свободным,  чтобы  я  не  могу  было  быть,  а  так  не  могла,  что  на  тебя  же  не  за  ними,  -  сказал  он.  -  Но 
Prompt: Что бы ни случилось, я всегда буду
Ответ модели: Что бы ни 

In [23]:
def try_to_del_var (name):
    if name in locals():
        del globals()[name]

In [24]:
try_to_del_var ("model")
try_to_del_var ("tokenizer")
try_to_del_var ("wrapped_tokenizer")
try_to_del_var ("trainer")

gc.collect()
torch.cuda.empty_cache()
torch.cuda.ipc_collect()
torch.cuda.synchronize()

# Post-train SFT

Для SFT-этапа можно использовать значительно меньше данных, поэтому возьмём модель крупнее. Рассмотрим базовую модель Qwen2.5-0.5B, с которой вы встречались в уроках. Обучите её генерировать ответы на инструктивные русскоязычные вопросы.

Для оценки качества используйте такой набор вопросов:


In [25]:
questions_rus = [
    "сколько планет в нашей солнечной системе?",
    "расскажи стих",
    "когда собирать крыжовник?",
    "Как быстро выучить новый язык?"
  ]

Подготовьте инструктивный датасет [d0rj/alpaca-cleaned-ru](https://huggingface.co/datasets/d0rj/alpaca-cleaned-ru) для обучения базовой модели. Представьте его в виде transformers.Dataset для диалоговых данных input=system, instruction=user, output=assistant.

In [26]:
ds = load_dataset(SFT_DATASET, split="train")
print ("изначальный датасет: ", ds)
if JUST_SMOKE_TEST:
    ds = ds.select(range(100))
else:
    # Вот оценка 501/51760 05:55 < 10:08:02. 10 часов на всём датасете - долго
    # оставим меньше:
    ds = ds.select(range(2000)) 
print ("итоговый датасет: ", ds)

изначальный датасет:  Dataset({
    features: ['input', 'instruction', 'output'],
    num_rows: 51760
})
итоговый датасет:  Dataset({
    features: ['input', 'instruction', 'output'],
    num_rows: 2000
})


In [27]:
ds[0]

{'input': '',
 'instruction': 'Дайте три совета, как оставаться здоровым.',
 'output': '1. Соблюдайте сбалансированную и питательную диету. Убедитесь, что в ваш рацион входят разнообразные фрукты и овощи, нежирный белок, цельнозерновые продукты и полезные жиры. Это помогает обеспечить ваш организм необходимыми питательными веществами для оптимального функционирования и может помочь предотвратить хронические заболевания.\n\n2. Занимайтесь регулярной физической активностью. Упражнения имеют решающее значение для поддержания крепких костей, мышц и здоровья сердечно-сосудистой системы. Старайтесь уделять не менее 150 минут умеренным аэробным упражнениям или 75 минут интенсивным упражнениям каждую неделю.\n\n3. Высыпайтесь. Достаточное количество качественного сна имеет решающее значение для физического и психического благополучия. Он помогает регулировать настроение, улучшать когнитивные функции и поддерживает здоровый рост и иммунную функцию. Старайтесь спать 7-9 часов каждую ночь.'}

In [28]:
def to_messages(example):
    return {
        "messages": [
            {"role": "system", "content": example.get("input", "") or "Вы — доброжелательный помощник."},
            {"role": "user", "content": example["instruction"]},
            {"role": "assistant", "content": example["output"]}
        ]
    }

ds = ds.map(to_messages)

print(ds[0]["messages"])

[{'content': 'Вы — доброжелательный помощник.', 'role': 'system'}, {'content': 'Дайте три совета, как оставаться здоровым.', 'role': 'user'}, {'content': '1. Соблюдайте сбалансированную и питательную диету. Убедитесь, что в ваш рацион входят разнообразные фрукты и овощи, нежирный белок, цельнозерновые продукты и полезные жиры. Это помогает обеспечить ваш организм необходимыми питательными веществами для оптимального функционирования и может помочь предотвратить хронические заболевания.\n\n2. Занимайтесь регулярной физической активностью. Упражнения имеют решающее значение для поддержания крепких костей, мышц и здоровья сердечно-сосудистой системы. Старайтесь уделять не менее 150 минут умеренным аэробным упражнениям или 75 минут интенсивным упражнениям каждую неделю.\n\n3. Высыпайтесь. Достаточное количество качественного сна имеет решающее значение для физического и психического благополучия. Он помогает регулировать настроение, улучшать когнитивные функции и поддерживает здоровый рост

С помощью trl SFTTrainer дообучите модель.

In [29]:
gc.collect()
torch.cuda.empty_cache()
torch.cuda.ipc_collect()
torch.cuda.synchronize()

In [30]:
print(torch.cuda.memory_summary())

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |   1527 MiB |   8055 MiB |  89103 GiB |  89102 GiB |
|       from large pool |   1527 MiB |   8021 MiB |  88838 GiB |  88836 GiB |
|       from small pool |      0 MiB |     52 MiB |    265 GiB |    265 GiB |
|---------------------------------------------------------------------------|
| Active memory         |   1527 MiB |   8055 MiB |  89103 GiB |  89102 GiB |
|       from large pool |   1527 MiB |   8021 MiB |  88838 GiB |

In [31]:
model_id = SFT_MODEL_NAME
tok = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype="auto")
cfg = SFTConfig(
    output_dir="full_sft",
    per_device_train_batch_size=1,  
    logging_steps=1,
    max_length=1024,
    num_train_epochs=1,
    lr_scheduler_type="cosine",
    learning_rate=5e-5,
    run_name='SFT'
)
trainer = SFTTrainer(
    model=model,
    args=cfg,
    train_dataset=ds,
    processing_class=tok,
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00, 17.00it/s]


In [32]:
%%time
trainer.train()
#trainer.save_model(FINETUNED_MODEL_PATH)
#tok.save_pretrained(FINETUNED_MODEL_PATH)
trainer.model.gradient_checkpointing_disable() # для борьбы с:
trainer.model.config.use_cache = True          # Caching is incompatible with gradient checkpointing in Qwen3DecoderLayer. Setting `past_key_value=None`.

Step,Training Loss
1,5.313100
2,2.095400
3,1.606700
4,1.475500
5,2.292700
6,0.981300
7,1.356700
8,1.568400
9,1.271300
10,1.062200


CPU times: user 23min 29s, sys: 2min 41s, total: 26min 11s
Wall time: 26min 19s


Посмотрите на адекватность генерации на финальных данных. 
Факты могут быть ошибочными, но язык ответа должен быть русским и должна сохраняться структура ответа.

In [33]:
def generate_answer(model, tok, question, max_new_tokens=200):
    messages = [
        {"role": "system", "content": "Вы — доброжелательный помощник."},
        {"role": "user", "content": question},
    ]

    text = tok.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    inputs = tok(text, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            pad_token_id=tok.pad_token_id,
            eos_token_id=tok.eos_token_id
        )

    answer = tok.decode(outputs[0], skip_special_tokens=True)

    if question in answer:
        answer = answer.split(question, 1)[-1].strip()

    return answer


In [34]:
%%time
for q in questions_rus:
    print("------------------------------")
    print(f"Вопрос: {q}\n")
    print(f"Ответ: {generate_answer(model, tok, q)}")

------------------------------
Вопрос: сколько планет в нашей солнечной системе?

Ответ: assistant
<think>

</think>

В нашей солнечной системе 8 планет: Меркурий, Венера, Земля, Меркурий, Меркурий, Меркурий и Юпитер, Сатурн, Уран и Плутон.
------------------------------
Вопрос: расскажи стих

Ответ: assistant
<think>

</think>

«Я вижу тебе, как ты приближается, и чувствую себя так, как будто ты меня обнимает. Ты держишься так близко, и я чувствую себя так хорошо, как будто ты моя. Ты так красив, и я люблю тебя. Я чувствую себя так, как будто ты меня любишь. Я чувствую себя так, как будто ты меня любишь. Я чувствую себя так, как будто ты меня любишь. Я чувствую себя так, как будто ты меня любишь. Я чувствую себя так, как будто ты меня любишь. Я чувствую себя так, как будто ты меня любишь. Я чувствую себя так, как будто ты меня любишь. Я чувствую себя так, как будто ты меня любишь. Я чувствую себя так, как
------------------------------
Вопрос: когда собирать крыжовник?

Ответ: assista

In [35]:
try_to_del_var ("model")
try_to_del_var ("tok")
try_to_del_var ("trainer")

gc.collect()
torch.cuda.empty_cache()
torch.cuda.ipc_collect()
torch.cuda.synchronize()

# Заключение

- Проведены эксперименты с pre-training и SFT.
- pre-trained модель смогла сгенерировать слова и предложения, похожие на Русский язык.
- finetuned модель ответила на вопросы с учётом небольшого времени выделенного на обучение на платном сервисе и на урезанный размер датасета.